In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"riditazamanadikta","key":"f0afc81d6ab5abed56a69d3a92026326"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d omkargurav/face-mask-dataset

 88% 143M/163M [00:01<00:00, 148MB/s]
100% 163M/163M [00:01<00:00, 129MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/face-mask-dataset.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
!ls

data  face-mask-dataset.zip  kaggle.json  sample_data


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
train_data_dir = '/content/0.png'
val_data_dir = '/content/2.png'

In [ ]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16
NUM_CLASSES = 2

In [ ]:
train_datagen = ImageDataGenerator
(
    rescale:=1.0/255,
    rotation_range:=20,
    width_shift_range:=0.2,
    height_shift_range:=0.2,
    shear_range:=0.2,
    zoom_range:=0.2,
    horizontal_flip:=True,
    fill_mode:='nearest'
)

(0.00392156862745098, 20, 0.2, 0.2, 0.2, 0.2, True, 'nearest')

In [ ]:
val_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
train_generator = train_datagen.flow_from_directory
(
    train_data_dir,
    target_size:=IMAGE_SIZE,
    batch_size:=BATCH_SIZE,
    class_mode:='categorical'
)

('/content/0.png', (224, 224), 16, 'categorical')

In [ ]:
val_generator = val_datagen.flow_from_directory
(
    val_data_dir,
    target_size:=IMAGE_SIZE,
    batch_size:=BATCH_SIZE,
    class_mode:='categorical'
)

('/content/2.png', (224, 224), 16, 'categorical')

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(NUM_CLASSES, activation='softmax')(x)

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
model = models.Model(inputs=base_model.input, outputs=predictions)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint('face_mask_detector.h5', save_best_only=True)

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=[checkpoint]
)

TypeError: ignored